In [ ]:
import pandas as pd
import os
import unicodedata

# --- CONFIGURAÇÕES ---
BASE_DIR = '/content/drive/MyDrive/criminalidade_df'
RAW_DIR = os.path.join(BASE_DIR, 'data/raw/SINESP')
PROCESSED_DIR = os.path.join(BASE_DIR, 'data/processed')

os.makedirs(PROCESSED_DIR, exist_ok=True)

# --- FUNÇÕES ---

def normalizar_texto(texto):
    """Remove acentos e coloca em minúsculas para facilitar filtros"""
    if not isinstance(texto, str): return str(texto)
    nfkd = unicodedata.normalize('NFKD', texto)
    return u"".join([c for c in nfkd if not unicodedata.combining(c)]).lower().strip()

def processar_sinesp(nome_arquivo):
    caminho_arquivo = os.path.join(RAW_DIR, nome_arquivo)
    print(f"📂 Lendo arquivo SINESP: {nome_arquivo}")

    try:
        if nome_arquivo.endswith('.csv'):
            # SINESP as vezes usa ; ou , dependendo da versão
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='utf-8', engine='python')
            if df.shape[1] < 2:
                df = pd.read_csv(caminho_arquivo, sep=',', encoding='utf-8')
        else:
            df = pd.read_excel(caminho_arquivo)

    except Exception as e:
        print(f"❌ Erro ao ler arquivo: {e}")
        return None

    # Normaliza colunas para evitar erros de Case Sensitive
    df.columns = [normalizar_texto(c) for c in df.columns]
    print(f"   Colunas encontradas: {df.columns.tolist()}")

    # 1. Filtro Geográfico: DF
    # Procurar coluna de UF (pode ser 'uf', 'sigla_uf', etc)
    col_uf = next((c for c in df.columns if c in ['uf', 'sigla_uf', 'unidade_da_federacao']), None)

    if not col_uf:
        print("❌ Coluna de UF não encontrada.")
        return None

    # Filtra DF (usando normalização para garantir)
    df['uf_norm'] = df[col_uf].apply(normalizar_texto)
    df_df = df[df['uf_norm'] == 'distrito federal'].copy()

    if df_df.empty:
        print("⚠️ Nenhum dado do DF encontrado neste arquivo.")
        return None

    # 2. Padronização de Colunas
    # O SINESP costuma ter: 'municipio', 'ano', 'mes', 'crime', 'vítimas'

    # Mapeamento dinâmico (tenta achar os nomes comuns)
    col_map = {
        col_uf: 'uf',
        # next((c for c in df_df.columns if 'municipio' in c), 'municipio'): 'municipio',
        next((c for c in df_df.columns if c == 'ano'), 'ano'): 'ano',
        next((c for c in df_df.columns if c == 'mes'), 'mes'): 'mes',
        next((c for c in df_df.columns if 'tipo' in c or 'crime' in c), 'tipo_crime'): 'tipo_crime',
        next((c for c in df_df.columns if 'vitima' in c or 'ocorrencia' in c or 'valor' in c), 'quantidade'): 'quantidade'
    }

    df_clean = df_df.rename(columns=col_map)[list(col_map.values())].copy()

    # 3. Limpeza de Valores
    # Crimes
    df_clean['tipo_crime'] = df_clean['tipo_crime'].str.title()

    # Quantidade (garantir numérico)
    df_clean['quantidade'] = pd.to_numeric(df_clean['quantidade'], errors='coerce').fillna(0).astype(int)

    # 4. Filtro de Período (Sua pesquisa: 2014-2025)
    df_clean = df_clean[df_clean['ano'].between(2014, 2025)]

    # Salva
    caminho_saida = os.path.join(PROCESSED_DIR, 'sinesp_seguranca_df.csv')
    df_clean.to_csv(caminho_saida, index=False)

    print(f"\n✅ SUCESSO! Base SINESP processada.")
    print(f"   Total de Registros (DF): {len(df_clean)}")
    print("   Crimes mais comuns:")
    print(df_clean.groupby('tipo_crime')['quantidade'].sum().sort_values(ascending=False).head())

    return df_clean




arquivo_sinesp = 'sinesp_dados_brutos.xlsx'

if os.path.exists(os.path.join(RAW_DIR, arquivo_sinesp)):
    processar_sinesp(arquivo_sinesp)
else:
    print(f"⚠️ Por favor, faça o upload do arquivo '{arquivo_sinesp}' na pasta: {RAW_DIR}")

📂 Lendo arquivo SINESP: sinesp_dados_brutos.xlsx
   Colunas encontradas: ['uf', 'tipo crime', 'ano', 'mes', 'ocorrencias']

✅ SUCESSO! Base SINESP processada.
   Total de Registros (DF): 864
   Crimes mais comuns:
tipo_crime
Furto De Veículo          40589
Roubo De Veículo          28496
Tentativa De Homicídio     5981
Estupro                    5170
Homicídio Doloso           3329
Name: quantidade, dtype: int64
